

### 파일 구조가 아래와 같이 정리되어 있을 때 각각의 클래스를 train, test set으로 나누고 tf record로 변환 시켜보기


file_name </br>
   * ㄴclass01 </br>
         -...png </br>
         -...png </br>
         -...png </br>
   * ㄴclass02 </br>
         -...png </br>
         -...png </br>
         -...png </br>
   * ㄴclass03 </br>
         -...png </br>
         -...png </br>
         -...png </br>

In [12]:
import os
import shutil #  파일 복사, 이동, 이름 변경, 디렉토리 생성하는 모듈
from sklearn.model_selection import train_test_split
import tensorflow as tf

### tf record file 생성

In [19]:
# 데이터 폴더 경로
data_dir = './tomato_leaf'
categories = ['bugs', 'dried', 'normal', 'spotted']   # 클래스명

# 데이터를 저장할 폴더 경로
output_dir = './tomato_leaf_split'
os.makedirs(output_dir, exist_ok=True)

# 각 카테고리별로 데이터를 분할해서 저장할 폴더 경로 생성
for category in categories:
    os.makedirs(os.path.join(output_dir, 'x_train', category), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'x_test', category), exist_ok=True)

# 데이터 분할 비율 설정 (예: 80% 학습 데이터, 20% 테스트 데이터)
train_ratio = 0.8

# 각 카테고리별로 데이터를 로드하고 분할해서 저장
for category in categories:
    # 카테고리 폴더 내의 파일 리스트 가져오기
    files = os.listdir(os.path.join(data_dir, category))
    # 파일 리스트를 학습 데이터와 테스트 데이터로 분할
    train_files, test_files = train_test_split(files, train_size=train_ratio, random_state=42)
    # 분할된 파일을 해당 폴더로 복사
    for file in train_files:
        shutil.copy(os.path.join(data_dir, category, file), os.path.join(output_dir, 'x_train', category))
    for file in test_files:
        shutil.copy(os.path.join(data_dir, category, file), os.path.join(output_dir, 'x_test', category))

print("file spilted")

# ----------------------------------------------------------------------

# TFRecord 파일 생성 함수
def create_tfrecord_file(input_dir, output_file):
    writer = tf.io.TFRecordWriter(output_file)

    for category in categories:
        category_dir = os.path.join(input_dir, category)
        for filename in os.listdir(category_dir):
            image_path = os.path.join(category_dir, filename)
            with open(image_path, 'rb') as f:
                image_data = f.read()

            feature = {
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_data])),
                'label': tf.train.Feature(bytes_list=tf.train.BytesList(value=[category.encode('utf-8')])),
            }

            example = tf.train.Example(features=tf.train.Features(feature=feature))
            serialized_example = example.SerializeToString()
            writer.write(serialized_example)

    writer.close()
    print(f'TFRecord file {output_file} created.')

# TFRecord 파일 생성
train_tfrecord_file = 'train.tfrecord'
test_tfrecord_file = 'test.tfrecord'
create_tfrecord_file(os.path.join(output_dir, 'x_train'), train_tfrecord_file)
create_tfrecord_file(os.path.join(output_dir, 'x_test'), test_tfrecord_file)


file spilted
TFRecord file train.tfrecord created.
TFRecord file test.tfrecord created.


### tf record  파일 읽어오기

In [20]:
import tensorflow as tf


# TFRecord 파일에서 데이터를 읽어오는 함수
def read_tfrecord_file(file_path):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.string),
    }

    def _parse_function(example_proto):
        example = tf.io.parse_single_example(example_proto, feature_description)
        image = tf.image.decode_image(example['image'])
        label = example['label']
        return image, label

    dataset = tf.data.TFRecordDataset(file_path)
    dataset = dataset.map(_parse_function)
    return dataset

# 생성된 TFRecord 파일에서 데이터를 읽어오기
train_tfrecord_file = 'train.tfrecord'
test_tfrecord_file = 'test.tfrecord'

train_dataset = read_tfrecord_file(train_tfrecord_file)
test_dataset = read_tfrecord_file(test_tfrecord_file)

# 데이터 확인
for image, label in train_dataset.take(5):  # 처음 5개 데이터만 확인
    # 이미지 시각화 또는 원하는 작업 수행
    # 예시로 이미지 shape과 레이블 출력
    print("Image Shape:", image.shape)
    print("Label:", label.numpy().decode('utf-8'))


Image Shape: (2419, 3226, 3)
Label: bugs
Image Shape: (400, 600, 3)
Label: bugs
Image Shape: (407, 612, 3)
Label: dried
Image Shape: (1080, 1628, 3)
Label: dried
Image Shape: (900, 1200, 3)
Label: normal
